# Notes on using latent Gaussian Processes for solving the inverse DLS problem

Vinothan N. Manoharan

Sources on DLS and inversion: 

- Thy Doan Mai Le's [writeup](https://github.com/thydmle/pydls/blob/master/code/writeup.pdf) on [pydls](https://github.com/manoharan-lab/pydls)
- Thy's 2020 APS March Meeting talk on &ldquo;Bayesian inference of particle size distributions in dynamic light scattering&rdquo;
- Chu, B. “Dynamic Light Scattering.” In Soft Matter Characterization, edited by Redouane Borsali and Robert Pecora, 335–72. Dordrecht: Springer Netherlands, 2008. http://www.springerlink.com/content/h8915g5ll7613n3m/.

Sources on Gaussian processes:
- Ebden, Mark. [&ldquo;Gaussian Processes: A Quick Introduction.&rdquo;](http://arxiv.org/abs/1505.02965">http://arxiv.org/abs/1505.02965) *ArXiv:1505.02965 [Math, Stat]*, May 12, 2015.
- Mackay, David J. C. [&ldquo;Introduction to Gaussian Processes.&rdquo;](http://inference.org.uk/mackay/gpB.pdf) In *Neural Networks and Machine Learning*, edited by Christopher Bishop. NATO ASI Series. Series F, Computer and Systems Sciences ; Vol. 168. Berlin ; New York: Springer, 1998.
- Rasmussen, Carl Edward, and Christopher K. I. Williams. [*Gaussian Processes for Machine Learning*](http://www.gaussianprocess.org/gpml/chapters/). Cambridge: MIT Press, 2006.
- [Kat Bailey's blog post on &ldquo;Gaussian processes for dummies&rdquo;](http://katbailey.github.io/post/gaussian-processes-for-dummies/)
- [Alex Bridgland's blog post &ldquo;Introduction to Gaussian Processes part 1&rdquo;](http://bridg.land/posts/gaussian-processes-1)
- [Example of Gaussian process regression on noisy data, using the Python package George](http://dfm.io/george/dev/tutorials/first/)
- [PyMC3 documentation on latent Gaussian processes](https://docs.pymc.io/notebooks/GP-Latent.html)

In [1]:
%matplotlib inline 
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm

## Problem statement

Given a measured autocorrelation function from a dynamic light scattering experiment, we would like to infer the size distribution of particles in the sample.  We assume that all the particles are made of the same material, so that they all have the same refractive index.  We also assume that they are spherical, dilute, and non-interacting. 

### Background: autocorrelation function for monodisperse particles

In DLS we measure the intensity autocorrelation function.  The unnormalized version of the autocorrelation function is 
$$
G^{(2)}(q, \tau) = \left<I(q, t)I(q, t+\tau)\right>,
$$
where $\tau$ is the lag time, $I$ is the measured intensity, $q$ is the magnitude of the scattering wavevector, and the angle brackets denote a statistical average.  To derive a relationship between the measured autocorrelation function and the properties of the particles we are looking at, we need to start with the field autocorrelation function.  The magnitude of the field autocorrelation function is 
$$
\left|G^{(1)}(q, \tau)\right| = \left<E(q, t)E^*(q, t+\tau)\right>,
$$
where $E$ is the electric field and $E^*$ its complex conjugate.  We can also define a *normalized* versions of each autocorrelation function by dividing by the value of the function at zero lag time.  We use lowercase letters to denote the normalized versions.  The normalized intensity autocorrelation function is
$$
g^{(2)}(q, \tau) = \frac{\left<I(q, t)I(q, t+\tau)\right>}{\left<I(q, t)^2\right>},
$$
and the normalized field autocorrelation function is
$$
\left|g^{(1)}(q, \tau)\right| = \frac{\left<E(q, t)E^*(q, t+\tau)\right>}{\left<E(q, t)E^*(q, t)\right>}.
$$




The intensity autocorrelation function is related to the field autocorrelation function through the Siegert relation:
$$
g^{(2)}(q, \tau) = 1 + \beta\left|g^{(1)}(q, \tau)\right|^2,
$$
where $\beta$ is an instrumental parameter related to the coherence of the light.

For monodisperse spheres at very low concentration, the normalized field autocorrelation function is an exponential function of lag time:
$$
\left|g^{(1)}(q, \tau)\right| = \exp(-\Gamma \tau),
$$
where the linewidth $\Gamma(q) = Dq^2$, and $D$ is the diffusion coefficient.  We can use the Stokes-Einstein relation to relate the diffusion coefficient to the particle radius $R$:
$$
D(R) = \frac{k_B T}{6\pi \eta R},
$$
where $k_B T$ is the thermal energy and $\eta$ the viscosity of the fluid.  We then have a model for the intensity autocorrelation function for monodisperse Brownian spheres:
$$
g^{(2)}(q, \tau) = 1 + \beta\exp\left(-2D(R)q^2 \tau\right).
$$
To determine the particle size from a measurement of the autocorrelation function for monodisperse spheres, we fit the model above to the data.

---

> Note: my notation is a little different from the Boualem paper.  I use $D$ for the diffusion coefficient, whereas they use $D$ for the diameter.

---

### Background: autocorrelation function for polydisperse particles

To derive a model for the intensity autocorrelation function for polydisperse particles, we go back to the field autocorrelation function.  We assume that the particle size distribution is continuous.  Then the normalized field autocorrelation function generalizes to
$$
\left|g^{(1)}(q, \tau)\right| = \int_0^\infty G(D) \exp\left(-D q^2 \tau\right)\,dD,
$$
where $G(D)$ is the (intensity-weighted) distribution of diffusion coefficients.  For a discrete distribution, the relation is
$$
\left|g^{(1)}(q, \tau)\right| = \sum_i G(D_i) \exp\left(-D_i q^2 \tau\right).
$$
Essentially, each particle species has its own decay time, and the autocorrelation function is just the sum of all the exponential decays.  Each decay is weighted by a factor $G(D)$. In both cases, the distribution $G(D)$ is normalized. We can see this by taking the limit as $\tau\to 0$, in which case the normalized function becomes unity.  Taking this limit on the right-hand side as well gives
$$
1 = \int_0^\infty G(D)\,dD.
$$
We then have
$$
g^{(2)}(q, \tau) = 1 + \beta\left(\int_0^\infty G(D) \exp\left(-D q^2 \tau\right)\,dD\right)^2.
$$

### Intensity-weighted to number-weighted distribution 

We now have a model for the polydisperse intensity autocorrelation function.  In principle, we could fit for the distribution $G(D)$ using a nonparametric method.  But the distribution $G(D)$ is not always useful, because it is intensity-weighted, and the intensity depends on the scattering angle.  Let's say we were to infer $G(D)$ by fitting the above equation to data taken at a particular scattering angle.  If we were to change that scattering angle, we would get a different $G(D)$, for the exact same sample.  That's not a useful property for a distribution.

Alternatively, we can work with a number-weighted distribution instead.  Let $f(R)$ be the number-weighted distribution of particle radii.  Then we let
$$
G(D) = k f(R) C(R),
$$
where $k$ is a constant that does not depend on $R$, and $C(R)$ is the scattered intensity of a particle with diameter $R$ at the given scattering angle.  

---

> TODO: I (vnm) need to expand this section

---

In terms of the number-weighted distribution $f(R)$, the model for the intensity autocorrelation function is
$$
g^{(2)}(q, \tau) = 1 + \beta\left(k\int_0^\infty f(R)C(R) \exp\left(-D(R) q^2 \tau\right)\,dR\right)^2,
$$
where $D(R)$ is given by the Stokes-Einstein equation.

## Nonparametric fitting of the polydisperse intensity autocorrelation function

In a typical approach to infer the size distribution $f(R)$, we expand $f(R)$ in a set of basis functions:
$$ 
f(R) = \sum_j w_j \phi_j(R).
$$
where $\phi_j(R)$ is the $j$th basis function and $w_j$ is a weight.  We then infer the weights $w_j$ through a regularized or Bayesian approach.  Once we have $w_j$, $f(R)$ can be constructed using the above equation.  

In the simplest case, the basis functions are rectangles that are equally spaced in $R$.  This is essentially the Boualem approach, if I understand correctly.  They discretize the radii as $R_j$, and then they infer $f_j = f(R_j)$ for all the points in $f$.



But we can choose more complex basis functions if we wanted to.  The advantage of doing this is that we can incorporate different kinds of prior information by choosing different bases.  

## Latent Gaussian processes for nonparametric fitting

A more elegant way to describe $f(R)$ is as a Gaussian process.  A Gaussian process is the limit of a multidimensional Gaussian distribution as the number of dimensions approaches infinity.
$$
f(R) \sim \mathcal{GP}\left(m(R), k(R, R')\right), 
$$
where $m(R)$ is a mean function and $k(R, R')$ is a covariance function.  What this means is that if we choose a finite number of points, $R_1, R_2, \ldots, R_n$, then
$$ 
g(R_1), g(R_2), \ldots, g(R_n) 
$$
can be considered to be sampled from a finite-dimensional (multivariate) Gaussian.  This is the definition of a Gaussian process.  Formally, it is a collection of random variables, any finite number of which have a Gaussian distribution. 

Less formally, it is a _probability distribution on functions_.  The mean function is the generalization of the mean vector we would use in a multivariate Gaussian.  The covariance function is the generalization of the covariance matrix.  A sample from a Gaussian process is an entire _function_.  We can control how smooth the functions are through the covariance matrix.  

Another way to look at it is to consider expanding our $f(R)$ in terms of basis functions again.  If we assume a Gaussian prior on the $w_j$, and we take the number of basis functions to infinity, we end up with a Gaussian process.  It turns out that our choice of basis functions affects only the covariance function $k(R, R')$ (see Rasmussen and Williams).  So we've come up with a method to do a nonparametric fit, but using an infinite number of basis functions&mdash;and we don't have to choose them!  We simply choose a covariance function. That covariance function is also a _prior_.

### Mean and covariance functions

We assume that $m(R) = 0$, for simplicity.  If we don't have any physical model for the data, this is the only choice that makes sense.

The covariance function $k(R,R')$ relates each observation to each other.  It relates to how smooth the function is.  One common choice for $k(R, R')$ is the *squared exponential*:
$$
k(R, R') = \sigma_f^2 \exp\left[-\frac{(R-R')^2}{2 l^2}\right].
$$
Two points far from one another, such that $(R-R')\gg l$, have no covariance, whereas two points very close have the maximum value of the covariance, $\sigma_f^2$.  You can think of $\sigma_f^2$ as the uncertainty in the model. $l$ is the length scale of the correlation.

### Latent processes

Most of the sources you will read on Gaussian processes deal with fitting unknown functions directly to data.  Here we have a different problem. We are not trying to fit a functional form to the correlation function (we know the functional form already), but rather to the size distribution that gives rise to the correlation function. we must turn to *latent* Gaussian process regression. 

### Challenges

There are many python packages that can do latent Gaussian process regression, including [pymc3](https://docs.pymc.io/notebooks/GP-Latent.html) and [pyro](https://pyro.ai/examples/gplvm.html).  There are a few challenges, though:

1. We must figure out a way to do the integral in the model above.
2. We must constrain the GP to be nonnegative.  This may not be too difficult.  We can use a transformation of the form $f(r) = \exp(h(r))$ where $h(r)$ is the Gaussian process.
3. We must figure out a way to describe the priors on the distribution in terms of the covariance function.  This might not be too hard either.

So the main sticking point appears to be point 1.  If we can find a way to evaluate the forward model, we can use all the power of the variational inference and Monte Carlo methods in pymc3 to tackle this problem.  

## Exercises


### Exercise 1: simulate a correlation function for monodisperse particles

Below, write a function that simulates $g^{(2)}(\tau)$ (with noise) for monodisperse particles.  The arguments to your function should be an array of data points $\tau_i$, wavevector $q$, the prefactor $\beta$, the particle size $R$, the temperature $T$, the viscosity $\eta$, and the noise level $\sigma$.  Assume that the noise is Gaussian.  That is,
$$
g^{(2)}(q, \tau) = 1 + \beta\exp\left(-2D(R) q^2 \tau\right) + \epsilon_i.
$$
where $\epsilon_i$ is a Gaussian random variable with mean 0 and standard deviation $\sigma$.

In [2]:
# space for answer (add more cells if necessary)

Plot a few different simulated correlations functions for 30 nm polystyrene particles in water at room temperature.  Choose $\beta=1$ for simplicity, and assume a 90$^\circ$ scattering angle and 660 nm incident light.  Make sure your functions make sense.  Is the decay time what you think it should be?

In [3]:
# space for answer (add more cells if necessary)

### Exercise 2: simulate a correlation function for polydisperse particles

Same as above, but now your function should additionally take (as an input) an array corresponding to a discrete size distribution $F(R)$.  Ignore the prefactor $k$ and the scattering strength $C(R)$.  Your model should therefore be
$$
g^{(2)}(q, \tau) = 1 + \beta\left(\sum_i F(R_i) \exp\left(-D(R_i) q^2 \tau\right)\right)^2 + \epsilon_i,
$$
where, as before, $\epsilon_i$ is a Gaussian random variable with mean 0 and standard deviation $\sigma$.

In [4]:
# space for answer (add more cells if necessary)

Make some size distributions and plot some simulated correlation functions.  Start by making sure your simulated correlation functions look like the ones above when you have monodisperse particles.  Then try adding additional particle sizes and see how the correlation function changes.  Do the shapes of the correlation function make sense?

In [5]:
# space for answer (add more cells if necessary)

### Exercise 3: getting started with pymc3

Simulate a correlation function for 30 nm monodisperse polystyrene particles in water at room temperature.  Then write a pymc3 model to fit this data for the particle size.  Assume that you know all the other parameters ($q$, $\beta$, $T$, $\eta$, $\sigma$).  Run the pymc3 sampler (the NUTS sampler) to get a posterior distribution on the particle size.  Explore the effects of different priors on $R$.  

Notes:
1.  [This tutorial](https://docs.pymc.io/notebooks/getting_started.html) might be useful
2.  You can install pymc3 in Anaconda using "conda install pymc3"

In [6]:
# space for answer (add more cells if necessary)

Plot the simulated correlation function, the &ldquo;best fit&rdquo; from the HMC sampler (the MAP), and the uncertainty on the best fit from the sampler.

In [7]:
# space for answer (add more cells if necessary)

Now assume that you don't know the noise in the experiment $\sigma$, but you have some prior information about it.  Write a new pymc3 model that includes the noise level $\sigma$ as a parameter, and include a prior on it (the tutorial above might be helpful).  Then fit, as you did before, using NUTS.  Plot the posterior distributions of the particle size and the noise level.  How close were the fits to the true values?

### Exercise 4: fitting real data

We can use the pymc3 model above to fit real data for monodisperse particles.  Find some real data for the correlation function of a fairly monodisperse system (LaNell, I think you have some data for small gold particles and maybe also some data for wild-type viruses).  Load the data and plot the correlation function.  Make sure all the units are correct.

In [8]:
# space for answer (add more cells if necessary)

Now use your pymc3 model to fit the data to determine both $R$ and $\sigma$. The other parameters should be fixed.  You'll have to find the temperature and the scattering angle, which depend on the instrument.  You'll also have to look up the viscosity of water at the temperature of the experiment.   Plot your posterior distributions for both $R$ and $\sigma$.  Do they look reasonable?

In [9]:
# space for answer (add more cells if necessary)

Plot the measured correlation function, the &ldquo;best fit&rdquo; from the HMC sampler (the MAP), and the uncertainty on the best fit from the sampler.

In [10]:
# space for answer (add more cells if necessary)

### Exercise 5: a simple Gaussian process regression in pymc3

Now simulate a correlation function for a distribution of particle sizes.  Make a pymc3 model for the correlation function as a Gaussian process with a squared exponential covariance function (`ExpQuad` in pymc3).  Then fit the pymc3 model to the data for the hyperparameters of the covariance function **and** the noise level $\sigma$.  

Notes:
1. [This tutorial](https://pymc3-testing.readthedocs.io/en/rtd-docs/notebooks/GP-introduction.html) might be useful


In [11]:
# space for answer (add more cells if necessary)

Plot the posterior predictive distribution as shown at the end of example 1 of [the tutorial](https://pymc3-testing.readthedocs.io/en/rtd-docs/notebooks/GP-introduction.html)

In [12]:
# space for answer (add more cells if necessary)

### Exercise 6: upload answers to GitHub

Make a new branch in the [manoharan-lab/pydls repository](https://github.com/manoharan-lab/pydls) and upload your solutions there.